<https://github.com/PolymathicAI/xVal>


In [1]:
import os

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.95"

In [2]:
import jax.numpy as jnp  # Oddly works in colab to set gpu

arr = jnp.array([1, 2, 3])
arr.devices()

{cuda(id=0)}

In [3]:
import icecream
from icecream import ic

icecream.install()
ic_disable = False
if ic_disable:
    ic.disable()
ic.configureOutput(includeContext=True, contextAbsPath=True)

In [4]:
import os
import ast

from datetime import datetime as dt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import hephaestus as hp
import jax
import jax.numpy as jnp
import numpy as np
import optax
import pandas as pd
from flax.training import train_state
from icecream import ic
from jax import random
from flax import struct
from flax.training import checkpoints
from jax.tree_util import tree_flatten
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm, trange
from hephaestus.models.simple_time_series import SimpleDS

pd.options.mode.copy_on_write = True

2024-05-28 14:25:58.810198: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
def line2df(line, idx):
    data_rows = []
    line = ast.literal_eval(line)
    for i, time_step in enumerate(line["data"]):
        row = {"time_step": i}
        # Add position data for each planet
        for j, position in enumerate(time_step):
            row[f"planet{j}_x"] = position[0]
            row[f"planet{j}_y"] = position[1]
        data_rows.append(row)

    df = pd.DataFrame(data_rows)
    description = line.pop("description")
    step_size = description.pop("stepsize")
    for k, v in description.items():
        for k_prop, v_prop in v.items():
            df[f"{k}_{k_prop}"] = v_prop
    df["time_step"] = df["time_step"] * step_size
    df.insert(0, "idx", idx)

    return df

In [6]:
files = os.listdir("data")
if "planets.parquet" not in files:
    with open("data/planets.data") as f:
        data = f.read().splitlines()

        dfs = []
        for idx, line in enumerate(tqdm(data)):
            dfs.append(line2df(line, idx))
        print("Concatenating dfs...")
        df = pd.concat(dfs)
    df.to_parquet("data/planets.parquet")
else:
    df = pd.read_parquet("data/planets.parquet")

In [7]:
# Get min, mean, and max number of time steps
df.groupby("idx").count().time_step.agg(["min", "mean", "max"])

min     30.000000
mean    44.511656
max     59.000000
Name: time_step, dtype: float64

In [8]:
# Get train test split at 80/20
train_idx = int(df.idx.max() * 0.8)
train_df = df.loc[df.idx < train_idx].copy()
test_df = df.loc[df.idx >= train_idx].copy()
del df
train_ds = SimpleDS(train_df)
test_ds = SimpleDS(test_df)

In [9]:
def make_batch(ds: SimpleDS, start: int, length: int):
    data = []
    for i in range(start, length + start):
        data.append(ds[i])

    return jnp.array(data)


batch = make_batch(train_ds, 0, 4)

In [10]:
time_series_regressor = hp.simple_time_series.SimplePred(train_ds, d_model=64 * 4)

In [11]:
key = random.PRNGKey(0)
init_key, dropout_key = random.split(key)
vars = time_series_regressor.init(
    {"params": init_key, "dropout": dropout_key}, batch, deterministic=False
)
dropout_key, original_dropout_key = random.split(dropout_key)
x = time_series_regressor.apply(
    vars, batch, deterministic=False, rngs={"dropout": dropout_key}
)

ic| simple_time_series.py:219 in __call__()
    mask.shape: (4, 26, 1, 59, 59)
ic| simple_time_series.py:222 in __call__()
    col_embeddings.shape: (26, 256)
    numeric_inputs.shape: (4, 26, 59)
ic| simple_time_series.py:226 in __call__()
    "before swap": 'before swap'
    repeated_numeric_indices.shape: (59, 26)
ic| simple_time_series.py:229 in __call__()
    "after swap": 'after swap'
    repeated_numeric_indices.shape: (26, 59)
ic| simple_time_series.py:231 in __call__()
    "Embedding!!": 'Embedding!!'
    numeric_col_embeddings.shape: (26, 59, 256)
ic| simple_time_series.py:239 in __call__()
    "Retiling": 'Retiling'
    numeric_col_embeddings.shape: (4, 26, 59, 256)
ic| simple_time_series.py:244 in __call__()
    "Before Broadcast": 'Before Broadcast'
    numeric_inputs.shape: (4, 26, 59)
    numeric_col_embeddings.shape: (4, 26, 59, 256)
ic| simple_time_series.py:257 in __call__()
    "Before where": 'Before where'
    numeric_broadcast.shape: (4, 26, 59, 256)
    nan_mask.

In [12]:
key = jax.random.PRNGKey(0)
key1, key2 = jax.random.split(key)

# Create random arrays
numeric_inputs = jax.random.normal(key1, (4, 26, 59))
numeric_col_embeddings = jax.random.normal(key2, (4, 26, 59, 256))
# numeric_inputs.shape: (4, 26, 59)
# numeric_col_embeddings.shape: (4, 26, 59, 256)
(numeric_inputs[:, :, :, None] * numeric_col_embeddings).shape

(4, 26, 59, 256)

In [13]:
import jax
import jax.numpy as jnp

# Set a random seed for reproducibility
key = jax.random.PRNGKey(0)
key1, key2 = jax.random.split(key)

# Create random arrays
numeric_inputs = jax.random.normal(key1, (4, 26, 59))
numeric_col_embeddings = jax.random.normal(key2, (4, 26, 26, 256))

# Print shapes of the arrays
print("numeric_inputs.shape:", numeric_inputs.shape)
print("numeric_col_embeddings.shape:", numeric_col_embeddings.shape)

# Attempt to broadcast
try:
    numeric_broadcast = (
        numeric_inputs[:, :, :, None, None] * numeric_col_embeddings[:, :, None, :, :]
    )
    print("Broadcasting successful! Shape of result:", numeric_broadcast.shape)
except ValueError as e:
    print("Error during broadcasting:", e)

numeric_inputs.shape: (4, 26, 59)
numeric_col_embeddings.shape: (4, 26, 26, 256)
Broadcasting successful! Shape of result: (4, 26, 59, 26, 256)


In [14]:
time_series_regressor.tabulate(
    {"params": init_key, "dropout": dropout_key},
    batch,
    console_kwargs={"force_jupyter": True, "width": 120},
)

ic| simple_time_series.py:219 in __call__()
    mask.shape: (4, 26, 1, 59, 59)
ic| simple_time_series.py:222 in __call__()
    col_embeddings.shape: (26, 256)
    numeric_inputs.shape: (4, 26, 59)
ic| simple_time_series.py:226 in __call__()
    "before swap": 'before swap'
    repeated_numeric_indices.shape: (59, 26)
ic| simple_time_series.py:229 in __call__()
    "after swap": 'after swap'
    repeated_numeric_indices.shape: (26, 59)
ic| simple_time_series.py:231 in __call__()
    "Embedding!!": 'Embedding!!'
    numeric_col_embeddings.shape: (26, 59, 256)
ic| simple_time_series.py:239 in __call__()
    "Retiling": 'Retiling'
    numeric_col_embeddings.shape: (4, 26, 59, 256)
ic| simple_time_series.py:244 in __call__()
    "Before Broadcast": 'Before Broadcast'
    numeric_inputs.shape: (4, 26, 59)
    numeric_col_embeddings.shape: (4, 26, 59, 256)
ic| simple_time_series.py:257 in __call__()
    "Before where": 'Before where'
    numeric_broadcast.shape: (4, 26, 59, 256)
    nan_mask.

                                                   SimplePred Summary                                                   
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ path                  ┃ module                ┃ inputs                ┃ outputs               ┃ params               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│                       │ SimplePred            │ float32[4,59,26]      │ float32[4,26,59]      │                      │
├───────────────────────┼───────────────────────┼───────────────────────┼───────────────────────┼──────────────────────┤
│ TimeSeriesTransforme… │ TimeSeriesTransformer │ deterministic: False  │ float32[4,26,59,272]  │                      │
│                       │                       │ numeric_inputs:       │                       │                      │
│                       │                       │ float32[4,59,26]      │                       │                      │
├───────────────────────┼───────────────────────┼───────────────────────┼───────────────────────┼──────────────────────┤
│ TimeSeriesTransforme… │ Embed                 │ int32[26]             │ float32[26,256]       │ embedding:           │
│                       │                       │                       │                       │ float32[29,256]      │
│                       │                       │                       │                       │                      │
│                       │                       │                       │                       │ 7,424 (29.7 KB)      │
├───────────────────────┼───────────────────────┼───────────────────────┼───────────────────────┼──────────────────────┤
│ TimeSeriesTransforme… │ PositionalEncoding    │ float32[4,26,59,256]  │ float32[4,26,59,272]  │                      │
├───────────────────────┼───────────────────────┼───────────────────────┼───────────────────────┼──────────────────────┤
│ TimeSeriesTransforme… │ TransformerBlock      │ deterministic: False  │ float32[4,26,59,272]  │                      │
│                       │                       │ k:                    │                       │                      │
│                       │                       │ float32[4,26,59,256]  │                       │                      │
│                       │                       │ mask:                 │                       │                      │
│                       │                       │ float32[4,26,1,59,59] │                       │                      │
│                       │                       │ q:                    │                       │                      │
│                       │                       │ float32[4,26,59,272]  │                       │                      │
│                       │                       │ v:                    │                       │                      │
│                       │                       │ float32[4,26,59,272]  │                       │                      │
├───────────────────────┼───────────────────────┼───────────────────────┼───────────────────────┼──────────────────────┤
│ TimeSeriesTransforme… │ MultiHeadDotProductA… │ -                     │ float32[4,26,59,272]  │                      │
│                       │                       │ float32[4,26,59,272]  │                       │                      │
│                       │                       │ -                     │                       │                      │
│                       │                       │ float32[4,26,59,256]  │                       │                      │
│                       │                       │ -                     │                       │                      │
│                       │                       │ float32[4,26,59,272]  │                       │                      │
│      

'\n\n'

In [15]:
def calculate_memory_footprint(params):
    """Calculate total memory footprint of JAX model parameters and total
    number of parameters."""
    total_bytes = 0
    # Flatten the parameter tree structure into a list of arrays
    flat_params, _ = tree_flatten(params)
    for param in flat_params:
        # Calculate bytes: number of elements * size of each element
        bytes_per_param = param.size * param.dtype.itemsize
        total_bytes += bytes_per_param
    return total_bytes


def count_parameters(params):
    return sum(jnp.prod(jnp.array(p.shape)) for p in jax.tree_util.tree_leaves(params))


mem = calculate_memory_footprint(vars)
total_params = count_parameters(vars)


print(f"Memory of custom: {mem / 1e6:.2f} MB with {total_params:,} parameters")

Memory of custom: 3.24 MB with 809,889 parameters


In [16]:
batch.shape

(4, 59, 26)

In [17]:
def create_causal_mask(tensor: jnp.ndarray):
    """Create a causal mask to mask out future values."""
    mask = jnp.tril(jnp.ones((tensor.shape[0], tensor.shape[1])))
    return mask


def create_padding_mask(tensor: jnp.ndarray):
    """Create a padding mask to mask out padded values."""
    mask = jnp.isnan(tensor)
    return mask


def mask_array(tensor: jnp.array):
    """Create a mask for the tensor"""
    causal_mask = create_causal_mask(tensor)
    padding_mask = create_padding_mask(tensor)
    mask = jnp.logical_or(causal_mask, padding_mask)
    return mask

In [18]:
jnp.array(train_ds[0]).shape

(59, 26)

In [19]:
mts_root_key = random.PRNGKey(44)
mts_main_key, ts_params_key, ts_data_key = random.split(mts_root_key, 3)


def clip_gradients(gradients, max_norm):
    total_norm = jnp.sqrt(sum(jnp.sum(jnp.square(grad)) for grad in gradients.values()))
    scale = max_norm / (total_norm + 1e-6)
    clipped_gradients = jax.tree_map(
        lambda grad: jnp.where(total_norm > max_norm, grad * scale, grad), gradients
    )
    return clipped_gradients


def base_loss(inputs, outputs):
    # Create mask for nan inputs
    # inputs = jnp.swapaxes(inputs, 0, 1)
    inputs = jnp.swapaxes(inputs, 1, 2)
    nan_mask = jnp.isnan(inputs)
    inputs = jnp.where(nan_mask, jnp.zeros_like(inputs), inputs)
    outputs = jnp.where(nan_mask, jnp.zeros_like(outputs), outputs)

    raw_loss = optax.squared_error(outputs, inputs)
    masked_loss = jnp.where(nan_mask, 0.0, raw_loss)
    loss = masked_loss.sum() / (~nan_mask).sum()

    return loss


def calculate_loss(params, state, inputs, dataset: SimpleDS, dropout_key, mask_key):
    outputs = state.apply_fn(
        {"params": params},
        # hp.mask_tensor(inputs, dataset, prng_key=mask_key),
        inputs,
        rngs={"dropout": dropout_key},
        deterministic=False,
    )
    loss = base_loss(inputs, outputs)
    # Create mask for nan inputs

    return loss


@jax.jit
def train_step(state: train_state.TrainState, batch, base_key):
    dropout_key, mask_key, new_key = jax.random.split(base_key, 3)

    def loss_fn(params):
        return calculate_loss(params, state, batch, train_ds, dropout_key, mask_key)

    grad_fn = jax.value_and_grad(loss_fn)

    # (loss, individual_losses), grad = grad_fn(state.params)
    loss, grad = grad_fn(state.params)
    # grad = replace_nans(grad)
    # grad = clip_gradients(grad, 1.0)
    state = state.apply_gradients(grads=grad)

    return state, loss, new_key


def evaluate(params, state, inputs, dataset: SimpleDS, dropout_key, mask_key):
    outputs = state.apply_fn(
        {"params": params},
        # hp.mask_tensor(inputs, dataset, prng_key=mask_key),
        inputs,
        deterministic=True,
    )
    loss = base_loss(inputs, outputs)
    return loss


@jax.jit
def eval_step(state: train_state.TrainState, batch, base_key):
    mask_key, dropout_key, new_key = jax.random.split(base_key, 3)

    def loss_fn(params):
        return evaluate(params, state, batch, train_ds, dropout_key, mask_key)

    # (loss, individual_losses), grad = grad_fn(state.params)
    loss = loss_fn(state.params)
    return loss, new_key


def create_train_state(model, prng, batch, lr):
    init_key, dropout_key = random.split(prng)
    params = model.init(
        {"params": init_key, "dropout": dropout_key}, batch, deterministic=False
    )
    # optimizer = optax.chain(optax.adam(lr))
    optimizer = optax.chain(optax.clip_by_global_norm(0.4), optax.adam(lr))
    # optimizer_state = optimizer.init(params)
    return train_state.TrainState.create(
        apply_fn=model.apply,
        params=params["params"],
        tx=optimizer,
        # tx_state=optimizer_state,
    )


batch_size = 2
# batch = train_ds[0]
# state = create_train_state(time_series_regressor, mts_main_key, batch, 0.0001)
state = create_train_state(time_series_regressor, mts_main_key, batch, 0.0001)

ic| simple_time_series.py:219 in __call__()
    mask.shape: (4, 26, 1, 59, 59)
ic| simple_time_series.py

:222 in __call__()
    col_embeddings.shape: (26, 256)
    numeric_inputs.shape: (4, 26, 59)
ic| simple_time_series.py:226 in __call__()
    "before swap": 'before swap'
    repeated_numeric_indices.shape: (59, 26)
ic| simple_time_series.py:229 in __call__()
    "after swap": 'after swap'
    repeated_numeric_indices.shape: (26, 59)
ic| simple_time_series.py:231 in __call__()
    "Embedding!!": 'Embedding!!'
    numeric_col_embeddings.shape: (26, 59, 256)
ic| simple_time_series.py:239 in __call__()
    "Retiling": 'Retiling'
    numeric_col_embeddings.shape: (4, 26, 59, 256)
ic| simple_time_series.py:244 in __call__()
    "Before Broadcast": 'Before Broadcast'
    numeric_inputs.shape: (4, 26, 59)
    numeric_col_embeddings.shape: (4, 26, 59, 256)
ic| simple_time_series.py:257 in __call__()
    "Before where": 'Before where'
    numeric_broadcast.shape: (4, 26, 59, 256)
    nan_mask.shape: (4, 26, 59)
ic| simple_time_series.py:271 in __call__()
    "Before Positional Encoding": 'Before

In [20]:
writer_name = "RowsLast"

writer_time = dt.now().strftime("%Y-%m-%dT%H:%M:%S")
model_name = writer_time + writer_name
train_summary_writer = SummaryWriter("runs/" + model_name)


test_set_key = random.PRNGKey(4454)

train_data_loader = DataLoader(train_ds, batch_size=512, shuffle=True)
test_data_loader = DataLoader(test_ds, batch_size=512, shuffle=True)
batch_count = 0
base_key = random.PRNGKey(42)

# Disable IC for training
ic.disable()
for j in trange(1, desc=f"epochs for {train_summary_writer.log_dir}"):
    # arrs = train_data_loader()
    for i in tqdm(train_data_loader, leave=False, desc="batches"):
        # for i in trange(len(pre_train) // batch_size, leave=False):
        # for i in trange(len(pre_train) // batch_size //10, leave=False):
        # batch = make_batch(train_ds, i[0], 4)

        state, loss, base_key = train_step(state, jnp.array(i), base_key)
        if jnp.isnan(loss):
            raise ValueError("Nan Value in loss, stopping")
        batch_count += 1

        if batch_count % 1 == 0:
            train_summary_writer.add_scalar(
                "loss/loss", np.array(loss.item()), batch_count
            )
        if batch_count % 10 == 0:
            test_loss, base_key = eval_step(
                state, jnp.array(next(iter(test_data_loader))), base_key
            )
            train_summary_writer.add_scalar(
                "loss/test_loss", np.array(test_loss.item()), batch_count
            )

train_summary_writer.close()

epochs for runs/2024-05-28T14:26:10RowsLast:   0%|          | 0/1 [00:00<?, ?it/s]

batches:   0%|          | 0/196 [00:00<?, ?it/s]

Before Transform (512, 59, 26) (512, 26, 59)
Before Transform (512, 59, 26) (512, 26, 59)
Before Transform (159, 59, 26) (159, 26, 59)


In [29]:
import orbax

ckpts_dir = "ckpts"
absolute_ckpts_dir = os.path.abspath(os.path.join(ckpts_dir, model_name))
if not os.path.exists(absolute_ckpts_dir):
    os.makedirs(absolute_ckpts_dir)

orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
checkpoints.save_checkpoint(
    ckpt_dir=absolute_ckpts_dir,
    target=state,
    step=1,
    keep=3,
    overwrite=True,
    orbax_checkpointer=orbax_checkpointer,
)

'/home/ubuntu/environment/Hephaestus/ckpts/2024-05-28T14:26:10RowsLast/checkpoint_1'

In [28]:
import flax

flax.__version__

'0.8.4'

In [31]:
def return_results(state, dataset):
    outputs = state.apply_fn(
        {"params": state.params},
        # hp.mask_tensor(jnp.array([train_ds[0]]), dataset, prng_key=key),
        jnp.array([dataset[0]]),
        deterministic=True,
    )
    return outputs


# test_result = jnp.squeeze(
#     state.apply_fn({"params": state.params}, jnp.array([train_ds[0]]))
# )
test_result = return_results(state, train_ds)
test_result.shape

(1, 26, 59)

In [ ]:
jnp.squeeze(test_result).shape

(59, 26)

In [39]:
test_result = jnp.squeeze(test_result)
df_pred = pd.DataFrame(test_result.T)
df_actual = pd.DataFrame(train_ds[0])

In [40]:
df_pred.head().shape, train_df.head().shape

((5, 26), (5, 27))

In [51]:
np.isnan(test_ds[60]).all()

True

In [ ]:
outputs2 = state.apply_fn(
    {"params": state.params},
    # hp.mask_tensor(jnp.array([train_ds[0]]), dataset, prng_key=key),
    jnp.array([test_ds[0]]),
    deterministic=True,
)

In [41]:
df_pred.columns = train_df.drop("idx", axis=1).columns
df_actual.columns = train_df.drop("idx", axis=1).columns

In [42]:
df_actual

,time_step,planet0_x,planet0_y,planet1_x,planet1_y,planet2_x,planet2_y,planet0_m,planet0_a,planet0_e,...,planet3_x,planet3_y,planet3_m,planet3_a,planet3_e,planet4_x,planet4_y,planet4_m,planet4_a,planet4_e
0,0.000000,1.560060,-0.854437,0.720639,0.691729,0.944008,2.700632,1.312562,1.944263,1.897807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.465116,1.689858,-0.514359,0.333295,0.942289,0.681604,2.785811,1.312562,1.944263,1.897807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.930233,1.753589,-0.154209,-0.124995,0.992368,0.412951,2.845461,1.312562,1.944263,1.897807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.395349,1.748068,0.212022,-0.556775,0.831727,0.140540,2.879232,1.312562,1.944263,1.897807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.860465,1.673573,0.569904,-0.870579,0.494812,-0.133144,2.887018,1.312562,1.944263,1.897807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.325581,1.533811,0.905607,-1.000151,0.053178,-0.405638,2.868949,1.312562,1.944263,1.897807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2.790698,1.335526,1.206832,-0.918192,-0.399688,-0.674530,2.825381,1.312562,1.944263,1.897807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3.255814,1.087824,1.463513,-0.641980,-0.767958,-0.937474,2.756889,1.312562,1.944263,1.897807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3.720930,0.801365,1.668203,-0.229808,-0.973515,-1.192211,2.664256,1.312562,1.944263,1.897807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4.186047,0.487549,1.816168,0.231092,-0.972449,-1.436585,2.548458,1.312562,1.944263,1.897807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
df_pred

,time_step,planet0_x,planet0_y,planet1_x,planet1_y,planet2_x,planet2_y,planet0_m,planet0_a,planet0_e,...,planet3_x,planet3_y,planet3_m,planet3_a,planet3_e,planet4_x,planet4_y,planet4_m,planet4_a,planet4_e
0,-0.275785,1.624566,-1.040170,0.856995,0.640420,1.055685,2.420976,1.339148,1.996103,1.801024,...,-0.557341,-0.557341,-0.557341,-0.557341,-0.557341,-0.557341,-0.557341,-0.557341,-0.557341,-0.557341
1,0.610337,1.826505,-0.633310,0.381749,0.918817,0.800520,2.530633,1.278594,1.943145,1.778811,...,-0.334904,-0.334904,-0.334904,-0.334904,-0.334904,-0.334904,-0.334904,-0.334904,-0.334904,-0.334904
2,1.120801,1.911175,-0.166414,-0.001320,0.993635,0.550755,2.600532,1.329746,1.951691,1.814045,...,-0.213807,-0.213807,-0.213807,-0.213807,-0.213807,-0.213807,-0.213807,-0.213807,-0.213807,-0.213807
3,1.504955,1.846015,0.103131,-0.550376,0.726106,0.154059,2.586007,1.296282,1.976745,1.822388,...,-0.294288,-0.294288,-0.294288,-0.294288,-0.294288,-0.294288,-0.294288,-0.294288,-0.294288,-0.294288
4,1.901667,1.727902,0.445547,-0.901958,0.422010,-0.157523,2.503025,1.271516,2.016119,1.871018,...,-0.350616,-0.350616,-0.350616,-0.350616,-0.350616,-0.350616,-0.350616,-0.350616,-0.350616,-0.350616
5,2.383418,1.549261,0.822473,-1.049372,0.085469,-0.375929,2.467679,1.265662,2.002356,1.880801,...,-0.370029,-0.370029,-0.370029,-0.370029,-0.370029,-0.370029,-0.370029,-0.370029,-0.370029,-0.370029
6,2.899381,1.390603,1.211903,-0.957502,-0.323316,-0.587850,2.446859,1.302992,1.986130,1.876865,...,-0.379619,-0.379619,-0.379619,-0.379619,-0.379619,-0.379619,-0.379619,-0.379619,-0.379619,-0.379619
7,3.298524,1.207800,1.506604,-0.603343,-0.671405,-0.923241,2.425750,1.289824,1.922976,1.802652,...,-0.297875,-0.297875,-0.297875,-0.297875,-0.297875,-0.297875,-0.297875,-0.297875,-0.297875,-0.297875
8,3.715902,0.942231,1.613335,-0.163032,-0.833703,-1.181494,2.414449,1.302125,1.964018,1.827798,...,-0.114423,-0.114423,-0.114423,-0.114423,-0.114423,-0.114423,-0.114423,-0.114423,-0.114423,-0.114423
9,4.119270,0.619142,1.708214,0.195192,-0.901813,-1.426362,2.300129,1.288882,2.001032,1.896381,...,-0.138453,-0.138453,-0.138453,-0.138453,-0.138453,-0.138453,-0.138453,-0.138453,-0.138453,-0.138453


In [44]:
# Subtract the actual from the predicted
df_diff = df_pred - df_actual
df_diff

,time_step,planet0_x,planet0_y,planet1_x,planet1_y,planet2_x,planet2_y,planet0_m,planet0_a,planet0_e,...,planet3_x,planet3_y,planet3_m,planet3_a,planet3_e,planet4_x,planet4_y,planet4_m,planet4_a,planet4_e
0,-0.275785,0.064506,-0.185733,0.136355,-0.051310,0.111677,-0.279656,0.026586,0.051840,-0.096783,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.145220,0.136647,-0.118951,0.048454,-0.023472,0.118916,-0.255178,-0.033968,-0.001118,-0.118996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.190569,0.157586,-0.012206,0.123675,0.001267,0.137804,-0.244929,0.017184,0.007428,-0.083761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.109606,0.097947,-0.108890,0.006399,-0.105621,0.013519,-0.293225,-0.016280,0.032482,-0.075419,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.041202,0.054329,-0.124357,-0.031378,-0.072801,-0.024379,-0.383994,-0.041045,0.071857,-0.026789,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.057836,0.015450,-0.083134,-0.049221,0.032292,0.029709,-0.401270,-0.046900,0.058094,-0.017005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.108684,0.055077,0.005071,-0.039311,0.076372,0.086679,-0.378522,-0.009570,0.041867,-0.020942,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.042710,0.119976,0.043091,0.038637,0.096552,0.014233,-0.331139,-0.022737,-0.021287,-0.095155,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-0.005028,0.140867,-0.054868,0.066776,0.139812,0.010717,-0.249807,-0.010437,0.019755,-0.070008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-0.066777,0.131593,-0.107954,-0.035900,0.070636,0.010223,-0.248328,-0.023680,0.056769,-0.001426,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
model_name = f"big_train_{dt.now()}_"

current_dir = os.getcwd()

if not os.path.exists("./pre_trained_models/"):
    os.makedirs("./pre_trained_models/")

path = os.path.join(current_dir, "./pre_trained_models/")


ckpt_dir = f"./pre_trained_models/{model_name}"

# checkpoints.save_checkpoint(
#     ckpt_dir=path, target=state, step=batch_count, overwrite=True, prefix=model_name
# )